In [0]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Activation, Dense, Dropout, Flatten
from keras import backend as K

In [3]:
from google.colab import files
uploaded = files.upload()

Saving processed.csv to processed.csv


In [5]:
uploaded = files.upload()

Saving processed_target.csv to processed_target.csv


In [0]:
m = pd.read_csv('processed.csv')
m_target = pd.read_csv('processed_target.csv', header=None)

In [7]:
print(len(m))
print(len(m_target))

178665
178665


In [8]:
m.columns

Index(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '3', '4', '5', '6', '7', '8', '9',
       'ANDROID-BOOKS_AND_REFERENCE', 'ANDROID-COMMUNICATION',
       'ANDROID-DATING', 'ANDROID-ENTERTAINMENT', 'ANDROID-GAME_ACTION',
       'ANDROID-GAME_ADVENTURE', 'ANDROID-GAME_ARCADE', 'ANDROID-GAME_BOARD',
       'ANDROID-GAME_CARD', 'ANDROID-GAME_CASINO', 'ANDROID-GAME_CASUAL',
       'ANDROID-GAME_MUSIC', 'ANDROID-GAME_PUZZLE', 'ANDROID-GAME_RACING',
       'ANDROID-GAME_ROLE_PLAYING', 'ANDROID-GAME_SIMULATION',
       'ANDROID-GAME_SPORTS', 'ANDROID-GAME_STRATEGY', 'ANDROID-GAME_TRIVIA',
       'ANDROID-GAME_WORD', 'ANDROID-HEALTH_AND_FITNESS', 'ANDROID-LIFESTYLE',
       'ANDROID-MAPS_AND_NAVIGATION', 'ANDROID-MUSIC_AND_AUDIO',
       'ANDROID-NEWS_AND_MAGAZINES', 'ANDROID-PERSONALIZATION',
       'ANDROID-PHOTOGRAPHY', 'ANDROID-PRODUCTIVITY', 'ANDROID-SOCIAL',
       'ANDROID-SPORTS', 'ANDROID-TOOLS', 'ANDROID-TRAVEL_AND_LOCAL'

In [0]:
X = m
Y = m_target

In [10]:
input_len = len(X.columns)
print(input_len)

73


In [0]:
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )
  
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [12]:
# Model Params
batch_size = 100
epochs = 100

# create baseline_model
model = Sequential()
model.add(Dense(80, input_dim=input_len, kernel_initializer='RandomNormal', activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))
model.compile(loss=['mape'], optimizer='adam', metrics=[coeff_determination,root_mean_squared_error])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 80)                5920      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 10,001
Trainable params: 10,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
early_stopping = EarlyStopping(patience=15, verbose=1)
checkpointer = ModelCheckpoint(filepath=r'age_prediction.hdf5', 
                               verbose=1, 
                               save_best_only=True)

model.fit(X, Y, 
          batch_size=batch_size, 
          epochs = epochs, 
          verbose=1,
          shuffle=True,
          validation_split=0.1,
         callbacks=[early_stopping, checkpointer])

In [13]:
uploaded = files.upload()

Saving age_prediction.hdf5 to age_prediction.hdf5


In [0]:
model.load_weights(r'age_prediction.hdf5')
y_pred = model.predict(X)

In [23]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(mean_absolute_percentage_error(Y, y_pred))

19.57794442752429


In [0]:
from google.colab import files
files.download('age_prediction.hdf5')